# Message Authentication Codes

## Introduction

Message Authentication Codes (MACs) are used to address two main information security objectives:

- **Authentication**. 
    - Is the message received by Bob really from Alice?
- **Data Integrity** 
    - Has the received message been altered during transmission?

To verify these, Alice sends an appendix (also called a 'tag'), $MAC$, which is based on a previously shared secret key along with the message $m$ as $(m, MAC)$.

In class, we have focused on encryption for preserving secrecy, but MACs address the different issue of data integrity. (But we would often want both.)

There exists different kinds of MACs but we will be focusing on the more commonly used Hashed MAC (HMAC).

### MAC in Action: Verifying a message sent by Alice

Imagine that Alice and Bob share a keyword (e.g. Alohomora) and that Alice sends a message to Bob. We illustrate a situation where Bob confirms that the message was sent by Alice by using HMAC.

In [11]:
import hmac # Python has batteries included: it implements MACs!
import hashlib  # for chosing our hashfunction

In [12]:
def verify(alice_msg_digest, bob_msg_digest):
    
    if hmac.compare_digest(alice_msg_digest, bob_msg_digest):
        print("The message has been authenticated! The MAC is valid")
    else:
        print("The message has been altered! The MAC is invalid")

In [13]:
key = "Alohomora" # shared among Alice and Bob
key_bytes = key.encode()

alice_mac = hmac.new(key=key_bytes, digestmod=hashlib.sha256)
print(f"MAC_object name: {alice_mac.name}")

MAC_object name: hmac-sha256


Alice is a huge Harry Potter fan and wants to share a nice quote with Bob:

> It is our choices, Harry, that show what we truly are, far more than our abilities.
> - Albus Dumbledore, Harry Potter and the Chamber of Secrets

In [14]:
message = "It is our choices, Harry, that show what we truly are, far more than our abilities."
message

'It is our choices, Harry, that show what we truly are, far more than our abilities.'

We encode the plaintext in binary (more specifically, UTF-8) to send it using a computer:

In [15]:
message_bytes = bytes(message, encoding="utf-8")
print(message_bytes)
print(message_bytes.hex())

b'It is our choices, Harry, that show what we truly are, far more than our abilities.'
4974206973206f75722063686f696365732c2048617272792c20746861742073686f772077686174207765207472756c79206172652c20666172206d6f7265207468616e206f7572206162696c69746965732e


Alice then uses HMAC to generate a code and send that code to Bob together with the message

In [16]:
alice_mac.update(message_bytes)

In [17]:
alice_message_digest = alice_mac.digest()
print(alice_message_digest)
print(alice_message_digest.hex())

b'\x8cxV\xe5\x1d\xa1?AqC\x15\x1aI\x032\xb6\x1dI\xa7q\xae \xc0f\x8e;\xf0\xf7\x13\x9c\xb2\xb5'
8c7856e51da13f417143151a490332b61d49a771ae20c0668e3bf0f7139cb2b5


Bob then uses the message and the symmetric key to check if the message was sent by Alice:

In [18]:
bob_message_digest = hmac.digest(
    key=key_bytes,
    msg=message_bytes,
    digest=hashlib.sha256
)

verify(alice_message_digest, bob_message_digest)

The message has been authenticated! The MAC is valid


### MAC in Action: A message has been altered by Eve

Eve is a Draco Malfoy fan and wants to interfere on Alice's quote choice! She changes the content of the message while it goes through a channel.

In [19]:
altered_message = message.replace("choices", "blood status")
altered_message_bytes = bytes(altered_message, encoding="utf-8")

Bob receives the message and tries to check with HMAC using a symmetric key like in the last case:

In [20]:
bob_message_digest = hmac.digest(
    key=key_bytes,
    msg=altered_message_bytes,
    digest=hashlib.sha256
)

verify(alice_message_digest, bob_message_digest)

The message has been altered! The MAC is invalid


However, Bob finds that the message has been altered and that he cannot trust it!

## HMACs

The above example made use of HMACs which are one of the most commonly used MACs on the internet and can be found in the IPSec and TLS protocols for secure and authenticated communication over a network.

The Hashed MAC (HMAC), or keyed-hash message authentication code, was invented in 1996 by Mihir Bellare, Ran Canetti, and Hugo Krawczyk.

As the name indicates, a cryptographic hash function $H$ (such as MD5, SHA1, and SHA256) is used along with a secret symmetric key $K$ of general length of $512$ bits. 0s are appended to shorter keys. Longer keys can be hashed to obtain a shorter one.

<img src="img/hmac.png" width = 800>

The basic idea behind MACs is to attach a tag $H(K \Vert m)$ for authentication to the message. This construct is however vulnerable to length extension attacks for hash functions. So we prepend $K \oplus opad$ and hash again.

Alice then computes:

> $HMAC(m,K) = H((K \oplus opad) \Vert H((K \oplus ipad)\Vert m))$ 

where 
- $\Vert$ indicates concatenation (for example, $X\Vert Y$ denotes the string obtained by appending $Y$ to $X$ )
- ipad and opad are inner- and outer- paddings, 
- $K$ is the key, and 
- $H$ is a hash function.

The inner- and outer- paddings are binary strings of length $512$ expressed in hexadecimal:

> $opad = 5C5C5C\dots5C$, $ipad = 363636\dots36$

One advantage of using HMACs is that there is no need to modify the code of the hash function for the implementation.

Now that the HMAC has been computed, it can be sent along with the message $m$ (which can be either encrypted or unencrypted) to the receiver, Bob. Since Bob also knows $K$, they can use the received message to compute $HMAC(m,K)$ again and to compare their result to the HMAC sent by Alice. If the two HMACs agree, the message can be assumed to be authentic and from Alice. This is because:

- By collision resistance, if $m$ gets altered to $m'$, $H(K \oplus ipad \Vert m')$ should differ from $H(K \oplus ipad \Vert m)$. As a result, $HMAC(m, K) \neq HMAC (m', K)$. Hence, $HMAC(m,K)$ can be used to verify the integrity of the message received.
- Bob can be sure that Alice sent the message since it is computationally infeasible to compute a valid tag without knowledge of the key $K$.

## MACs vs Digital Signatures

While MACs and Digital Signatures are both used to verify the authenticity of a message and, to some extent, its sender, they differ a lot when it comes to how they are structured. 

### Type of key

MACs use a symmetric key system where both the sender and the receiver share the same key which is used for message authentication. On the other hand, with digital signatures, the key is tied to a person and each person signs the message they send with their unique key as a way of authentication.

<img src="img/MAC _vs_DS.jpg" width=800>

In the diagram above, Alice appends the message with a MAC using the key **k** which is then used by Bob to verify the integrity of the message and authenticate that it has been sent by someone on the communication channel. If Bob wanted to reply to the message they would use the same **k** to append a MAC to their message and send it to Alice. Hence, the key is shared between the sender and the receiver.

On the other hand, with a digital signature, Alice sends the message signed by their **unique** signature which uses the key **k<sub>a</sub>**. This signature is used by Bob to verify both the authenticity of the message as well as ascertain that the sender is Alice specifically. If Bob were to reply to the message, they would sign their reply using their own **unique** signature which uses the key **k<sub>b</sub>**. As each sender has a separate key that is used to sign the message, the keys are not shared in the case of a digital signature.
<br>
Since, as discussed before, for MACs to work, both Alice and Bob need to be aware of the key. This gives rise to the problem of how to share the key without Eve finding it. The key needs to be encrypted but it is not enough for Bob to receive a message that is encrypted sufficiently against intrusion. Bob also needs to authenticate that it is Alice that sent the encrypted message so Alice signs the message with their Digital signature that can be used by Bob to verify the authenticity of the message. Once Bob has the MAC key, all future communication can be securely authenticated using the MAC.

### Why use MAC instead of always using Digital Signatures

In the case of digital signatures, the security of the signature rests on its computational difficulty. Since the signature itself is made public, the only way to ensure that it is authentic is by making it unfeasible to reverse engineer the signing method (the **d<sub>a</sub> and the primes p and q** in case of RSA). It is this computational complexity that makes communication using digital signatures and public-key encryption so time and resource consuming. On the other hand, in the case of MAC or symmetric key encryption, The security is derived from keeping the structure and information regarding the key itself a secret rather than computationally complex. Thus, while communication with the key in hand is quick and resource-friendly, any intruder mustn't get to know any information regarding the key itself as it may not be that computationally complex to find the entire key from a part of it.

# Galois Counter Mode

Galois Counter Mode (GCM) is a block cipher mode of operation built upon hashing and finite fields. It has become popular because it can combine Message Authentication Codes and Encryption in a single step, which is more performant than calcualting encryption and MAC separately. It is also capable of being a standalone MAC without any changes.

A diagram overview of GCM ([McGrew & Viega, 2004](https://csrc.nist.rip/groups/ST/toolkit/BCM/documents/proposedmodes/gcm/gcm-revised-spec.pdf)) from its specification is:

<img src="img/gcm_diagram.png" width=550>

The output of the function defined in the diagram is $GHASH(H, A, C)$, where $H = E_{k}(000 \ldots 0)$ is the hash of the 128-bit bit representation of zero, $A$ is the authenticated text, and $C$ is the ciphertext.

We note that GCM operates on $\mathbb{F}_{128}$ with $f(x) = 1 + x + x^2 + x^7 + x^{128}$, hence the multiplications shown in the diagram are in $\mathbb{F}_{128}$.

## MACs and Replay Attacks

A **replay attack** is one of the main challenges of key distribution. Eve may record/intercept a message from Alice and delay or resend it to Bob at a later time to trigger a response from Bob. Thus, we need a method to make sure that that each message is fresh.  This can be achieved in the following ways:

- *Sequence numbers*. A unique sequence number is attached to each message sent from Alice to Bob. If Bob ever sees the same sequence number again, they can conclude that the message received is a replay. Both Alice and Bob have to keep track of the sequence numbers which have already been used.

- *Timestamps*. Alice appends the current time to their message. To determine the validity of the message, Bob verifies that the included timestamp is within a predefined window of time. Both Alice and Bob need to have clocks set to the same time.

- *Nonces*. Nonces are one-time, random messages used as part of a challenge-response mechanism. This mechanism involves Alice sending a message containing a nonce to Bob and requiring Bob to send back a correct response to Alice's nonce. Nonces sometimes include a timestamp for added security (but this again requires clock synchronisation).

MACs use deterministic hash functions, and are hence vulnerable to replay attacks: if we have the same key and the same message, we send the same MAC. Since MACs do not incorporate any notion of state in their verification algorithms, we cannot check if the message is fresh. So, MACs alone cannot be used to defend against replay attacks but can be used in conjunction with the above methods.

### References

- W. Trappe, L.C. Washington, *Introduction to cryptography: with coding theory*, Pearson Education, 2020.
- [Wellesley College, *MACs: Message authentication and integrity*, Fall 2016.](http://cs.wellesley.edu/~cs310/lectures/11_MAC_slides_handouts.pdf)
- McGrew, David, and Viega, John. *The Galois/counter mode of operation (GCM)*. Submission to NIST Modes of Operation Process 20 (2004) (https://csrc.nist.rip/groups/ST/toolkit/BCM/documents/proposedmodes/gcm/gcm-revised-spec.pdf)